In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle

image_dict = pickle.load( open( "image_dict.pickle", "rb" ) )
one_hot_score_dict =  pickle.load( open( "one_hot_score_dict.pickle", "rb" ) )


In [4]:
#df = pd.read_csv('ign_with_img_urls.csv')
#%store -r


In [5]:
image_dict

{0: array([[[21, 33, 45],
         [21, 33, 45],
         [24, 34, 46],
         ..., 
         [24, 35, 53],
         [24, 35, 53],
         [24, 35, 53]],
 
        [[21, 33, 45],
         [21, 33, 45],
         [23, 33, 45],
         ..., 
         [24, 35, 53],
         [24, 35, 53],
         [24, 35, 53]],
 
        [[20, 32, 44],
         [21, 33, 45],
         [23, 33, 45],
         ..., 
         [24, 35, 53],
         [24, 35, 53],
         [25, 36, 54]],
 
        ..., 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 9, 10, 15],
         [ 9,  9, 17],
         [ 9,  9, 17]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 9, 10, 15],
         [ 9, 10, 15],
         [ 7, 10, 15]],
 
        [[ 0,  0,  0],
         [ 0,  0,  0],
         [ 0,  0,  0],
         ..., 
         [ 9, 10, 15],
         [ 9, 10, 15],
         [ 7, 10, 15]]], dtype=uint8), 1: array([[[21, 33, 45],
         

In [44]:
print len(image_dict) #we have 16913 images
#get list of unique keys
list_of_keys = image_dict.keys()
#select a random 2,913 indices
random_shuffle_keys = list_of_keys
from random import shuffle
shuffle(random_shuffle_keys)
#print random_shuffle_keys[0:10]
#split the data of both the image arrays, and the one_hots
#get the last 2913 keys
test_keys = random_shuffle_keys[-2913:]
train_keys = random_shuffle_keys[0:-2913]
print len(test_keys)
print len(train_keys)


16913
2913
14000


In [45]:
one_hot_score_dict[50]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]

In [7]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

In [8]:
n_classes = 11 #we have 11 classes, score 0 to 10
batch_size = 100 #we'll start basic here
image_size = 160 #shape of image is 160x160x3

In [9]:
x = tf.placeholder('float', [None, 76800])
y = tf.placeholder('float')


In [10]:

def neural_network_model(data):
    hidden_1_layer = {'weights' : tf.Variable(tf.random_normal([76800, n_nodes_hl1])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights' : tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights' : tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights' : tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases': tf.Variable(tf.random_normal([n_classes]))}
    # (input_data * weights) + biases
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1) #relu is our activation function (rectified linear)

    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    #output layer functions a little differently
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']

    return output


In [63]:
def train_neural_network(x):
    prediction = neural_network_model(x) #get that one hot array
    #now let's actually compare the results
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, y))
    # learning rate default is 0.001 on the adamoptimizer
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    #How many epochs?
    #cycles is feed forward + back prop
    hm_epochs = 10

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int(len(train_keys)/batch_size)):
                keys_for_batch = train_keys[(i*batch_size):(i+1)*batch_size]
                epoch_x = np.array([image_dict[value].flatten() for value in keys_for_batch])
                epoch_y = np.array([one_hot_score_dict[value] for value in keys_for_batch])
                #print 'Size of batch: ' + str(len(epoch_x))
                i, c = sess.run([optimizer, cost], feed_dict = {x: epoch_x, y: epoch_y})
                epoch_loss += c
            print 'Epoch ' + str(epoch+1) + 'completed out of' + str(hm_epochs), 'loss:' + str(epoch_loss)
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        pred_label = tf.argmax(prediction, 1)
        corr_convert = tf.argmax(y,1)
        print 'Accuracy: ' + str(accuracy.eval({x:np.array([image_dict[value].flatten() for value in test_keys])
                                                          , y:np.array([one_hot_score_dict[value] for value in test_keys
                                                                       ])}))
        for i in range(len(test_keys)):
            val = test_keys[i]
            #feed_dict = {x: np.array([image_dict[value].flatten()])}
            classification = pred_label.eval( {x: np.array([image_dict[val].flatten()])})
            class_list[val] = classification[0]
            #correct_list[i] = corr_convert.eval({y:np.array([one_hot_score_dict[value]])})[0]

In [64]:
class_list = {}
train_neural_network(x)

Epoch 1completed out of10 loss:1768230443.0
Epoch 2completed out of10 loss:1244398675.75
Epoch 3completed out of10 loss:944069216.0
Epoch 4completed out of10 loss:704055508.875
Epoch 5completed out of10 loss:537723539.375
Epoch 6completed out of10 loss:639289518.75
Epoch 7completed out of10 loss:595809623.125
Epoch 8completed out of10 loss:474216129.875
Epoch 9completed out of10 loss:391371416.312
Epoch 10completed out of10 loss:389437996.188
Accuracy: 0.272228


In [53]:
#standard guess
correct_list = {}

sum_of_squares = 0
for i in range(len(test_keys)):
    value = test_keys[i]
    sum_of_squares += (np.argmax(one_hot_score_dict[value]) - 8)**2
print sum_of_squares

11704


In [72]:
#using the neural network
sum_of_squares = 0
for i in range(len(test_keys)):
    value = test_keys[i]
    actual_answer = np.argmax(one_hot_score_dict[value])
    sum_of_squares += (actual_answer - class_list[value])**2
print sum_of_squares

11704


In [73]:
class_list

{8193: 8,
 8194: 8,
 16388: 8,
 16385: 8,
 16394: 8,
 2: 8,
 14185: 8,
 16: 8,
 16401: 8,
 20: 8,
 8218: 8,
 16412: 8,
 8221: 8,
 30: 8,
 16416: 8,
 8225: 8,
 35: 8,
 16421: 8,
 39: 8,
 8232: 8,
 16428: 8,
 16430: 8,
 16433: 8,
 51: 8,
 16439: 8,
 16440: 8,
 57: 8,
 13663: 8,
 8252: 8,
 16445: 8,
 16451: 8,
 70: 8,
 16455: 8,
 16456: 8,
 75: 8,
 76: 8,
 81: 8,
 82: 8,
 16469: 8,
 87: 8,
 88: 8,
 17764: 8,
 90: 8,
 16476: 8,
 16477: 8,
 16478: 8,
 8288: 8,
 98: 8,
 100: 8,
 5478: 8,
 17: 8,
 8299: 8,
 108: 8,
 8301: 8,
 8302: 8,
 111: 8,
 8308: 8,
 15038: 8,
 129: 8,
 133: 8,
 8327: 8,
 8330: 8,
 16524: 8,
 8333: 8,
 2755: 8,
 8341: 8,
 151: 8,
 8348: 8,
 158: 8,
 16544: 8,
 16545: 8,
 163: 8,
 164: 8,
 169: 8,
 16554: 8,
 16557: 8,
 8367: 8,
 16560: 8,
 16562: 8,
 180: 8,
 183: 8,
 8378: 8,
 8379: 8,
 8380: 8,
 2762: 8,
 16574: 8,
 8383: 8,
 16576: 8,
 193: 8,
 8386: 8,
 16580: 8,
 198: 8,
 16584: 8,
 16587: 8,
 8396: 8,
 207: 8,
 8401: 8,
 9827: 8,
 9494: 8,
 16597: 8,
 16601: 8,
 166

In [12]:
#for i in range(len(train_keys)/batch_size):
#    #print i
#    keys_for_batch = train_keys[(i*batch_size):(i+1)*batch_size]
#    print len(keys_for_batch)

In [13]:
#with open('image_dict.pickle', 'wb') as f:
#    pickle.dump(image_dict, f)

In [14]:
#with open('one_hot_score_dict.pickle', 'wb') as f:
#    pickle.dump(one_hot_score_dict, f)